# Chapter 7 Bilingual PySpark: Blending Python and SQL code

This chapter covers

- Drawing a parallel between PySpark’s instruction set and the SQL vocabulary
- Registering data frames as temporary views or tables to query them using Spark SQL
- Using the catalog to create, reference, and delete registered tables for SQL querying
- Translating common data manipulations instructions from Python to SQL, and vice versa
- Using SQL-style clauses inside certain PySpark methods

# Start a pyspark session

In [47]:
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
import pyspark.sql.functions as F
import pyspark.sql.types as T
import helper_functions
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import numpy as np
warnings.filterwarnings("ignore")

# change the account name to your email account
account='sli'

# define a root path to access the data in the DataAnalysisWithPythonAndPySpark
root_path='/net/clusterhn/home/'+account+'/isa460/data/'

# check if the Spark session is active. If it is activate, close it

try:
    if spark:
        spark.stop()
except:
    pass    

spark = (SparkSession.builder.appName("Bilingual PySpark: Blending Python and SQL code")
        .config("spark.port.maxRetries", "100")
        .getOrCreate())

# confiture the log level (defaulty is WARN)
spark.sparkContext.setLogLevel('ERROR')

In [3]:
from helper_functions import displayByGroup

elements = spark.read.csv(
    root_path+"elements/Periodic_Table_Of_Elements.csv",
    header=True,
    inferSchema=True)

displayByGroup(elements, 5)

+------------+---------+------+----------+----------------+---------------+
|AtomicNumber|Element  |Symbol|AtomicMass|NumberofNeutrons|NumberofProtons|
+------------+---------+------+----------+----------------+---------------+
|1           |Hydrogen |H     |1.007     |0               |1              |
|2           |Helium   |He    |4.002     |2               |2              |
|3           |Lithium  |Li    |6.941     |4               |3              |
|4           |Beryllium|Be    |9.012     |5               |4              |
|5           |Boron    |B     |10.811    |6               |5              |
+------------+---------+------+----------+----------------+---------------+
only showing top 5 rows

+-----------------+------+-----+-----+-----------+-------+
|NumberofElectrons|Period|Group|Phase|Radioactive|Natural|
+-----------------+------+-----+-----+-----------+-------+
|1                |1     |1    |gas  |null       |yes    |
|2                |1     |18   |gas  |null       |yes  

# Register a data frame as a SQL table

In [4]:
elements.createOrReplaceTempView("elements_t")

In [6]:
query="select period, count(*) from elements_t where phase='liq' group by period"
spark.sql(query).show(5)

+------+--------+
|period|count(1)|
+------+--------+
|     6|       1|
|     4|       1|
+------+--------+



## Using the Spark catalog to manage SQL table/view

In [7]:
spark.catalog

In [8]:
# list all sql tables
spark.catalog.listTables()

[Table(name='elements_t', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [9]:
# drop sql table/view
spark.catalog.dropTempView("elements_t")

True

# Using SQL-style expressions in PySpark

we will use a public data set provided by Backblaze, which provided hard-drive data and statistics. Backblaze is a company that provides cloud storage and backup. Since 2013, they have provided data on the drives in their data center, and over time have moved to a focus on failures and diagnosis. To get the files, you can download them from the website (http://mng.bz/4jZa). We will use the latest data in June 2024.

In [10]:
df=spark.read.csv('/opt/shared/backblaze/', header=True, inferSchema=True)

In [11]:
df.count()

1202433

## Identify the model having the highest failure rate.

In [12]:
# select only the useful columns for our query
selected_columns=['date',
 'serial_number',
 'model',
 'capacity_bytes',
 'failure']


df1=df.select(selected_columns)

df1.show(10, False)

+----------+--------------+--------------------+--------------+-------+
|date      |serial_number |model               |capacity_bytes|failure|
+----------+--------------+--------------------+--------------+-------+
|2024-06-29|PL1331LAGXN3AH|HGST HDS5C4040ALE630|4000787030016 |0      |
|2024-06-29|PL2331LAGSTSYJ|HGST HMS5C4040ALE640|4000787030016 |0      |
|2024-06-29|S2ZYJ9FG405079|ST500LM012 HN       |500107862016  |0      |
|2024-06-29|Z3056CGY      |ST4000DM000         |4000787030016 |0      |
|2024-06-29|Z305FT10      |ST4000DM000         |4000787030016 |0      |
|2024-06-29|Z305FX0R      |ST4000DM000         |4000787030016 |0      |
|2024-06-29|Z305G7YZ      |ST4000DM000         |4000787030016 |0      |
|2024-06-29|Z305G8KA      |ST4000DM000         |4000787030016 |0      |
|2024-06-29|Z305G8M6      |ST4000DM000         |4000787030016 |0      |
|2024-06-29|Z305GBZ7      |ST4000DM000         |4000787030016 |0      |
+----------+--------------+--------------------+--------------+-

In [13]:
# get driver capacity in gigabytes

df2=df1.selectExpr("model", "capacity_bytes/power(1024,3) capacity_GB", "date", "failure")

df2.show()

+--------------------+-----------------+----------+-------+
|               model|      capacity_GB|      date|failure|
+--------------------+-----------------+----------+-------+
|HGST HDS5C4040ALE630|3726.023277282715|2024-06-29|      0|
|HGST HMS5C4040ALE640|3726.023277282715|2024-06-29|      0|
|       ST500LM012 HN|465.7617416381836|2024-06-29|      0|
|         ST4000DM000|3726.023277282715|2024-06-29|      0|
|         ST4000DM000|3726.023277282715|2024-06-29|      0|
|         ST4000DM000|3726.023277282715|2024-06-29|      0|
|         ST4000DM000|3726.023277282715|2024-06-29|      0|
|         ST4000DM000|3726.023277282715|2024-06-29|      0|
|         ST4000DM000|3726.023277282715|2024-06-29|      0|
|         ST4000DM000|3726.023277282715|2024-06-29|      0|
|         ST4000DM000|3726.023277282715|2024-06-29|      0|
|         ST4000DM000|3726.023277282715|2024-06-29|      0|
|         ST4000DM000|3726.023277282715|2024-06-29|      0|
|         ST4000DM000|3726.023277282715|

In [19]:
# calcualte failure rate by model

df3=df2.filter(F.col('failure').isNotNull()).groupBy('model', 'capacity_GB', 'failure').count().orderBy('model', 'capacity_GB')

df4=df3.groupBy('model', 'capacity_GB').pivot('failure').sum('count')

df3.show()

df4.show()

+--------------------+--------------------+-------+------+
|               model|         capacity_GB|failure| count|
+--------------------+--------------------+-------+------+
|      CT250MX500SSD1|  232.88591766357422|      0|  1817|
|         DELLBOSS VD|  447.06915283203125|      0|   140|
|HGST HDS5C4040ALE630|   3726.023277282715|      0|   120|
|HGST HDS724040ALE640|   3726.023277282715|      0|    30|
|HGST HMS5C4040ALE640|-9.31322574615478...|      0|     2|
|HGST HMS5C4040ALE640|   3726.023277282715|      0| 22747|
|HGST HMS5C4040BLE640|   3726.023277282715|      0|287823|
|HGST HMS5C4040BLE640|   3726.023277282715|      1|     2|
|HGST HMS5C4040BLE641|   3726.023277282715|      0|    30|
|HGST HUH721010ALE600|              9314.0|      0|    20|
|HGST HUH721212ALE600|             11176.0|      0|  3640|
|HGST HUH721212ALE604|-9.31322574615478...|      0|    40|
|HGST HUH721212ALE604|             11176.0|      1|    14|
|HGST HUH721212ALE604|             11176.0|      0| 7204

In [20]:
df5=df4.fillna(0).withColumn('failure_rate', F.col('1')/(F.col('0')+F.col('1'))).orderBy(F.desc('failure_rate'))

df5.show()

+--------------------+--------------------+------+---+--------------------+
|               model|         capacity_GB|     0|  1|        failure_rate|
+--------------------+--------------------+------+---+--------------------+
|Micron 5300 MTFDD...|  447.13167572021484|    11|  1| 0.08333333333333333|
|         ST4000DM000|-9.31322574615478...|    18|  1| 0.05263157894736842|
| WDC WUH721816ALE6L4|             14902.0|    86|  2|0.022727272727272728|
| TOSHIBA MG08ACA16TA|             14902.0|  3216| 13|0.004026014245896562|
|     WUH721816ALE6L4|             14902.0|   921|  1|0.001084598698481...|
| TOSHIBA MQ01ABF050M|   465.7617416381836|  2148|  2|9.302325581395349E-4|
|       ST16000NM001G|             14902.0| 24918| 22|8.821170809943865E-4|
|  TOSHIBA MQ01ABF050|   465.7617416381836|  3690|  2|5.417118093174431E-4|
|         ST4000DM000|   3726.023277282715| 77579| 22|2.835015012693135E-4|
|       ST500LM012 HN|   465.7617416381836|  3784|  1|2.642007926023778E-4|
|HGST HUH728

In [21]:
# is there a correlation between capacity and failure_rate

df5.select(F.corr('capacity_GB', 'failure_rate')).show()

+-------------------------------+
|corr(capacity_GB, failure_rate)|
+-------------------------------+
|           -0.09440025284627401|
+-------------------------------+



## SQL and Pyspark code comparison

In [22]:
df=spark.read.csv('/opt/shared/backblaze/', header=True, inferSchema=True)

selected_columns=['date',
 'serial_number',
 'model',
 'capacity_bytes',
 'failure']


backblaze=df.select(selected_columns)

backblaze.createOrReplaceTempView("backblaze_t")

### display the top 3 models that have the highest max capacity


In [23]:
# SQL way
spark.sql(
    """SELECT
           model,
           min(capacity_bytes / pow(1024, 3)) min_GB,
           max(capacity_bytes/ pow(1024, 3)) max_GB
        FROM backblaze_t
        GROUP BY 1
        ORDER BY 3 DESC"""
).show(5)

+-------------------+--------------------+-------+
|              model|              min_GB| max_GB|
+-------------------+--------------------+-------+
|    WUH721816ALE6L4|-9.31322574615478...|14902.0|
|TOSHIBA MG08ACA16TE|             14902.0|14902.0|
|      ST16000NM001G|-9.31322574615478...|14902.0|
|WDC WUH721816ALE6L4|             14902.0|14902.0|
|TOSHIBA MG08ACA16TA|-9.31322574615478...|14902.0|
+-------------------+--------------------+-------+
only showing top 5 rows



In [24]:
#Pyspark way

backblaze.groupby(F.col("model")).agg(
    F.min(F.col("capacity_bytes") / F.pow(F.lit(1024), 3)).alias("min_GB"),
    F.max(F.col("capacity_bytes") / F.pow(F.lit(1024), 3)).alias("max_GB"),
).orderBy(F.col("max_GB"), ascending=False).show(5) 

+-------------------+--------------------+-------+
|              model|              min_GB| max_GB|
+-------------------+--------------------+-------+
|    WUH721816ALE6L4|-9.31322574615478...|14902.0|
|TOSHIBA MG08ACA16TE|             14902.0|14902.0|
|      ST16000NM001G|-9.31322574615478...|14902.0|
|WDC WUH721816ALE6L4|             14902.0|14902.0|
|TOSHIBA MG08ACA16TA|-9.31322574615478...|14902.0|
+-------------------+--------------------+-------+
only showing top 5 rows



### display the top 3 models that have the highest max capacity, remove model that min capacity is equal to max capacity

filtering after grouping using having

In [25]:
# SQL way
spark.sql(
    """SELECT
           model,
           min(capacity_bytes / pow(1024, 3)) min_GB,
           max(capacity_bytes/ pow(1024, 3)) max_GB
        FROM backblaze_t
        GROUP BY 1
        HAVING min_GB!=max_GB
        ORDER BY 3 DESC"""
).show(5)

+-------------------+--------------------+-------+
|              model|              min_GB| max_GB|
+-------------------+--------------------+-------+
|      ST16000NM001G|-9.31322574615478...|14902.0|
|    WUH721816ALE6L4|-9.31322574615478...|14902.0|
|TOSHIBA MG08ACA16TA|-9.31322574615478...|14902.0|
|      ST16000NM000J|-9.31322574615478...|14902.0|
|TOSHIBA MG07ACA14TA|  0.1303899995982647|13039.0|
+-------------------+--------------------+-------+
only showing top 5 rows



In [26]:
#Pyspark way

backblaze.groupby(F.col("model")).agg(
    F.min(F.col("capacity_bytes") / F.pow(F.lit(1024), 3)).alias("min_GB"),
    F.max(F.col("capacity_bytes") / F.pow(F.lit(1024), 3)).alias("max_GB"),
).where(F.col("min_GB") != F.col("max_GB")).orderBy(F.col("max_GB"), ascending=False).show(5) 

+-------------------+--------------------+-------+
|              model|              min_GB| max_GB|
+-------------------+--------------------+-------+
|      ST16000NM001G|-9.31322574615478...|14902.0|
|    WUH721816ALE6L4|-9.31322574615478...|14902.0|
|TOSHIBA MG08ACA16TA|-9.31322574615478...|14902.0|
|      ST16000NM000J|-9.31322574615478...|14902.0|
|TOSHIBA MG07ACA14TA|  0.1303899995982647|13039.0|
+-------------------+--------------------+-------+
only showing top 5 rows



## Creating new tables/views using the CREATE keyword

In [27]:
spark.sql(
    """
    CREATE OR REPLACE TEMP VIEW drive_days AS
        SELECT model, count(*) AS drive_days
        FROM backblaze_t
        GROUP BY model"""
)
 
spark.sql(
    """CREATE OR REPLACE TEMP VIEW failures AS
           SELECT model, count(*) AS failures
           FROM backblaze_t
           WHERE failure = 1
           GROUP BY model"""
)

DataFrame[]

In [28]:
spark.catalog.listTables()

[Table(name='backblaze_t', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='drive_days', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='failures', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

### Adding data to our table using UNION and JOIN

In [29]:
backblaze_failure=backblaze.filter(F.col('failure')==1)
backblaze_working=backblaze.filter(F.col('failure')==0)

backblaze_failure.createOrReplaceTempView('backblaze_failure_t')
backblaze_working.createOrReplaceTempView('backblaze_working_t')

In [30]:
# sql way
spark.sql("""
        create or replace temp view backblaze_complete as
        select * from backblaze_failure_t union all
        select * from backblaze_working_t
""")

#pyspark way

backblaze_complet=backblaze_failure.union(backblaze_working)

In [31]:
# left join
spark.sql(
    """select
           drive_days.model,
           drive_days,
           failures
    from drive_days
    left join failures on drive_days.model = failures.model"""
).show(5)

+--------------------+----------+--------+
|               model|drive_days|failures|
+--------------------+----------+--------+
|         ST4000DM000|     77620|      23|
|         ST8000DM005|       683|    null|
| TOSHIBA MQ01ABF050M|      2150|       2|
|        ST8000NM0055|    311236|      35|
|Seagate BarraCuda...|        30|    null|
+--------------------+----------+--------+
only showing top 5 rows



### Organizing your SQL code better through subqueries and common table expressions

In [32]:
spark.sql("select * from backblaze_t").show()

+----------+--------------+--------------------+--------------+-------+
|      date| serial_number|               model|capacity_bytes|failure|
+----------+--------------+--------------------+--------------+-------+
|2024-06-29|PL1331LAGXN3AH|HGST HDS5C4040ALE630| 4000787030016|      0|
|2024-06-29|PL2331LAGSTSYJ|HGST HMS5C4040ALE640| 4000787030016|      0|
|2024-06-29|S2ZYJ9FG405079|       ST500LM012 HN|  500107862016|      0|
|2024-06-29|      Z3056CGY|         ST4000DM000| 4000787030016|      0|
|2024-06-29|      Z305FT10|         ST4000DM000| 4000787030016|      0|
|2024-06-29|      Z305FX0R|         ST4000DM000| 4000787030016|      0|
|2024-06-29|      Z305G7YZ|         ST4000DM000| 4000787030016|      0|
|2024-06-29|      Z305G8KA|         ST4000DM000| 4000787030016|      0|
|2024-06-29|      Z305G8M6|         ST4000DM000| 4000787030016|      0|
|2024-06-29|      Z305GBZ7|         ST4000DM000| 4000787030016|      0|
|2024-06-29|      Z305GC1B|         ST4000DM000| 4000787030016| 

In [33]:
# Finding drive models with highest failure rates using subqueries

spark.sql(
    """
    SELECT
        failures.model,
        failures / drive_days failure_rate
    FROM (
        SELECT
            model,
            count(*) AS drive_days
        FROM backblaze_t
        GROUP BY model) drive_days
    INNER JOIN (
        SELECT
            model,
            count(*) AS failures
        FROM backblaze_t
        WHERE failure = 1
        GROUP BY model) failures
    ON
        drive_days.model = failures.model
    ORDER BY 2 desc
    """
).show(5)

+--------------------+--------------------+
|               model|        failure_rate|
+--------------------+--------------------+
|Micron 5300 MTFDD...| 0.08333333333333333|
| WDC WUH721816ALE6L4|0.022727272727272728|
| TOSHIBA MG08ACA16TA|0.004024767801857585|
|     WUH721816ALE6L4|0.001082251082251...|
| TOSHIBA MQ01ABF050M|9.302325581395349E-4|
+--------------------+--------------------+
only showing top 5 rows



In [34]:
# Finding highest failure rates using common table expressions/CTE
spark.sql(
    """
    WITH drive_days as (            
        SELECT                       
            model,                   
            count(*) AS drive_days   
        FROM backblaze_t            
        GROUP BY model),             
    failures as (                    
        SELECT                      
            model,                   
            count(*) AS failures    
        FROM backblaze_t            
        WHERE failure = 1           
        GROUP BY model)              
    SELECT
        failures.model,
        failures / drive_days failure_rate
    FROM drive_days
    INNER JOIN failures
    ON
        drive_days.model = failures.model
    ORDER BY 2 desc
    """
).show(5)


+--------------------+--------------------+
|               model|        failure_rate|
+--------------------+--------------------+
|Micron 5300 MTFDD...| 0.08333333333333333|
| WDC WUH721816ALE6L4|0.022727272727272728|
| TOSHIBA MG08ACA16TA|0.004024767801857585|
|     WUH721816ALE6L4|0.001082251082251...|
| TOSHIBA MQ01ABF050M|9.302325581395349E-4|
+--------------------+--------------------+
only showing top 5 rows



In [35]:
# Finding the highest failure rate using Python scope rules

def failure_rate(df):
    drive_days = df.groupby(F.col("model")).agg(   
        F.count(F.col("*")).alias("drive_days")
    )
    failures = (
       df.where(F.col("failure") == 1)
        .groupby(F.col("model"))
        .agg(F.count(F.col("*")).alias("failures"))
    )
    answer = (                                               
        drive_days.join(failures, on="model", how="inner")
        .withColumn("failure_rate", F.col("failures") / F.col("drive_days"))
        .orderBy(F.col("failure_rate").desc())
    )
    return answer

In [36]:
failure_rate(backblaze).show()

+--------------------+----------+--------+--------------------+
|               model|drive_days|failures|        failure_rate|
+--------------------+----------+--------+--------------------+
|Micron 5300 MTFDD...|        12|       1| 0.08333333333333333|
| WDC WUH721816ALE6L4|        88|       2|0.022727272727272728|
| TOSHIBA MG08ACA16TA|      3230|      13|0.004024767801857585|
|     WUH721816ALE6L4|       924|       1|0.001082251082251...|
| TOSHIBA MQ01ABF050M|      2150|       2|9.302325581395349E-4|
|       ST16000NM001G|     24947|      22|8.818695634745661E-4|
|  TOSHIBA MQ01ABF050|      3692|       2|5.417118093174431E-4|
|         ST4000DM000|     77620|      23|2.963153826333419E-4|
|       ST500LM012 HN|      3785|       1|2.642007926023778E-4|
|HGST HUH728080ALE600|      3954|       1|2.529084471421345...|
|HGST HUH721212ALE604|     72101|      14|1.941720641877366...|
|       ST12000NM0008|     12020|       2|1.663893510815307...|
|        ST8000NM0055|    311236|      3

In [37]:
def most_reliable_drive_for_capacity(data, capacity_GB=2048, precision=0.25, top_n=3):
    """Returns the top 3 drives for a given approximate capacity.
 
    Given a capacity in GB and a precision as a decimal number, we keep the N
    drives where:
 
    - the capacity is between (capacity * 1/(1+precision)), capacity * (1+precision)
    - the failure rate is the lowest
 
    """
    capacity_min = capacity_GB / (1 + precision)
    capacity_max = capacity_GB * (1 + precision)
 
    answer = (
        data.where(f"capacity_GB between {capacity_min} and {capacity_max}")
        .orderBy("failure_rate", "capacity_GB", ascending=[True, False])
        .limit(top_n)                                                     
     )
 
    return answer

In [38]:
backblaze1=backblaze.withColumn('capacity_GB', F.col("capacity_bytes") / F.pow(F.lit(1024), 3))

most_reliable_drive_for_capacity(df5, capacity_GB=11176.0).show()

+-------------------+-----------+----+---+------------+
|              model|capacity_GB|   0|  1|failure_rate|
+-------------------+-----------+----+---+------------+
|      ST14000NM001G|    13039.0|4716|  0|         0.0|
|      ST14000NM000J|    13039.0|  67|  0|         0.0|
|WDC WUH721414ALE6L4|    13039.0|3605|  0|         0.0|
+-------------------+-----------+----+---+------------+



#Summary

- Spark provides a SQL API for data manipulation. This API supports ANSI SQL.
- Spark (and PySpark, by extension) borrows a lot of vocabulary and expected functionality from the way SQL manipulates tables. This is especially evident since the data manipulation module is called pyspark.sql.
- PySpark’s data frames need to be registered as views or tables before they can be queried with Spark SQL. You can give them a different name than the data frame you’re registering.
- PySpark’s own data frame manipulation methods and functions borrow SQL functionality, for the most part. Some exceptions, such as union(), are present and documented in the API.
- Spark SQL queries can be inserted in a PySpark program through the spark.sql function, where spark is the running SparkSession.
- Spark SQL table references are kept in a Catalog, which contains the metadata for all tables accessible to Spark SQL.
- PySpark will accept SQL-style clauses in where(), expr(), and selectExpr(), which can simplify the syntax for complex filtering and selection.
- When using Spark SQL queries with user-provided input, be careful with sanitizing the inputs to avoid potential SQL injection attacks.

## In class exercise

display average failure rate of driver by day, order by day, and visualize the result.

Use both SQL and pyspark to get the answer. Which one you prefer?

In [52]:
df=spark.read.csv('/opt/shared/backblaze/', header=True, inferSchema=True)

selected_columns=['date',
 'serial_number',
 'model',
 'capacity_bytes',
 'failure']


backblaze=df.select(selected_columns)

backblaze.createOrReplaceTempView("backblaze_t")

## PySpark way

## SQL way

## visualize the result